In [1]:
include("../../src/JWAS.jl")

JWAS

In [6]:
using DataFrames
using JWAS.Datasets
using JWAS

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [7]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

In [8]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [28]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [29]:
model_equations = "BW = intercept + age + sex + Animal"
                  # CW = intercept + age + sex + Animal";

In [30]:
model    = build_model(model_equations,R[1,1]);
#model    = build_model(model_equations,R);

In [31]:
set_covariate(model,"age","sex")
#set_covariate(model,"sex")

In [35]:
for i in model.covVec
    println(string(i))
end

age
sex


In [52]:
a=[string(i) for i in model.covVec]

2-element Array{Any,1}:
 "age"
 "sex"

In [57]:
@printf("Variables %-10s are set to be covariates.",model.covVec)


Variables [:age,:sex] are set to be covariates.

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [10]:
ped = get_pedigree(pedfile);

Finished!


In [11]:
set_random(model,"Animal", ped,G[1,1])

In [12]:
set_random(model,"age", 1.0,df=8)

In [13]:
outputMCMCsamples(model,"age")

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [14]:
out2 = runMCMC(model,data,methods="BayesC",chain_length=1000,printout_frequency=500,output_samples_frequency=10);

MCMC Information:
methods                                      BayesC
chain length                                   1000
estimatePi                                    false
constraint                                    false
missing_phenotypes                            false
starting value                                false
output sample frequency                          10
printout frequency                              500

Degree of freedom for hyper-parametes:
residual variances:                           4.000
iid random effect variances:                  8.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000



running MCMC for BayesC...  0%|                         |  ETA: 0:03:19
Posterior means at iteration: 500
Residual variance: 312.227887
Polygenic effects covariance matrix 
[23.828]
running MCMC for BayesC... 50%|████████████             |  ETA: 0:00:01
Posterior means at iteration: 1000
Residual variance: 303.726

In [15]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "MCMC samples for: 1:age"
  "MCMC samples for residual variance"
  "MCMC samples for polygenic effects var-cov parameters"
  "MCMC samples for: variance of 1:age"
  "Posterior mean of location parameters"

In [18]:
a=split("age sex"," ",keep=false)

2-element Array{SubString{ASCIIString},1}:
 "age"
 "sex"

In [19]:
[a;a]

4-element Array{SubString{ASCIIString},1}:
 "age"
 "sex"
 "age"
 "sex"